In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
#pip install utm
import utm

In [5]:
df_estaciones = pd.DataFrame([])
for id in np.arange(11, 40):
    page = requests.get("https://sinca.mma.gob.cl/index.php/estacion/index/key/D" +  np.str(id))

    soup = BeautifulSoup(page.content, 'html.parser')

    import re
    for element in soup.find_all(text=re.compile("Sistema")):
        #print(element)
        ele_split = element.split()
        if (ele_split[0] == 'Estación'):
            for element_2 in soup.find_all(text=re.compile("en línea")):
                df_estaciones = df_estaciones.append(
                    pd.DataFrame({'estacion_id': id, 'estacion': element.split('-')[0][:-1]}, index=[0]), ignore_index=True)

In [8]:
df_estaciones

,estacion_id,estacion
0,11,Estación Independencia
1,12,Estación La Florida
2,13,Estación Las Condes
3,14,Estación Parque O'Higgins
4,15,Estación Pudahuel
5,17,Estación El Bosque
6,18,Estación Cerro Navia
7,27,Estación Puente Alto
8,28,Estación Talagante
9,30,Estación QUILICURA


In [9]:
df_estaciones = df_estaciones.set_index('estacion_id')
for id in df_estaciones.index:
    page = requests.get("https://sinca.mma.gob.cl/index.php/estacion/index/key/D" +  np.str(id))

    soup = BeautifulSoup(page.content, 'html.parser')

    import re
    for element in soup.find_all(text=re.compile(" E ")):
        ele_split = element.split()
        UTM_E = np.float(ele_split[0])
        UTM_N = np.float(ele_split[2])
        df_estaciones.loc[id,'UTM_E'] = UTM_E
        df_estaciones.loc[id,'UTM_N'] = UTM_N
        lat,long = utm.to_latlon(UTM_E, UTM_N, 19, northern= False)
        df_estaciones.loc[id,'lat'] = lat
        df_estaciones.loc[id,'lon'] = long

In [10]:
df_estaciones = df_estaciones.reset_index()
df_estaciones

,estacion_id,estacion,UTM_E,UTM_N,lat,lon
0,11,Estación Independencia,346488.0,6300681.0,-33.422261,-70.651139
1,12,Estación La Florida,352504.0,6290304.0,-33.516667,-70.588161
2,13,Estación Las Condes,358305.0,6305906.0,-33.376776,-70.523257
3,14,Estación Parque O'Higgins,345673.0,6296019.0,-33.464176,-70.660703
4,15,Estación Pudahuel,337311.0,6298809.0,-33.437785,-70.750145
5,17,Estación El Bosque,345313.0,6286825.0,-33.547016,-70.666163
6,18,Estación Cerro Navia,338984.0,6299360.0,-33.433070,-70.732056
7,27,Estación Puente Alto,352049.0,6282013.0,-33.591356,-70.594431
8,28,Estación Talagante,318945.0,6272298.0,-33.673819,-70.952997
9,30,Estación QUILICURA,339594.0,6308625.0,-33.349631,-70.723846


In [11]:
import csv
df_estaciones[['estacion_id','estacion','lat','lon']].to_csv('Estaciones_Aire_RM.csv',index=False, quoting=csv.QUOTE_ALL, encoding='utf8')